In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartModel
from torch.optim import AdamW
from torch import nn
import os
from tqdm import tqdm

# Set random seed for reproducibility
torch.manual_seed(77)
np.random.seed(77)

# Step 1: Load and preprocess the data
def load_and_preprocess_data(file_path, is_test=False):
    df = pd.read_csv(file_path)
    print(f"Loaded data with {len(df)} rows and {len(df.columns)} columns")


    # Map test dataset columns to training feature names
    df.drop(columns = 'horse', inplace =True)

    column_mapping = {
        'horse_name': 'horse',
        'surface_code': 'surface_x',
        'purse': 'purse_x',
        'odds': 'dollar_odds',
        'sex': 'sex_code'
    }
    df = df.rename(columns=column_mapping)

    # Convert distance from yards to furlongs (1 furlong = 220 yards)
    if 'distance' in df.columns:
        df['distance_f'] = df['distance'] / 220.0

    # Define features
    features = [
        'horse', 'jockey', 'trainer', 'program_number', 'surface_x', 'distance_f', 'purse_x',
        'track_code', 'race_date', 'race_number', 'race_type', 'weight', 'age', 'sex_code',
        'dollar_odds', 'recent_finish_pos_1', 'recent_lengths_back_finish_1', 'recent_purse_1',
        'recent_finish_pos_2', 'recent_lengths_back_finish_2', 'recent_purse_2',
        'recent_finish_pos_3', 'recent_lengths_back_finish_3', 'recent_purse_3',
        'recent_finish_pos_4', 'recent_lengths_back_finish_4', 'recent_purse_4',
        'recent_finish_pos_5', 'recent_lengths_back_finish_5', 'recent_purse_5'
    ]
    targets = ['official_finish', 'speed_rating', 'win_time']

    # Add missing features for test dataset
    for i in range(1, 6):
        finish_pos_col = f'recent_finish_pos_{i}'
        lengths_col = f'recent_lengths_back_finish_{i}'
        purse_col = f'recent_purse_{i}'
        if finish_pos_col not in df.columns:
            df[finish_pos_col] = df.get(f'recentFinishPosition{i}', -1)
        if lengths_col not in df.columns:
            df[lengths_col] = -1
        if purse_col not in df.columns:
            df[purse_col] = -1

    # Drop rows with missing target values (only for training data)
    if not is_test and all(col in df.columns for col in targets):
        df = df.dropna(subset=targets).copy()
        print(f"After dropping NaN targets: {len(df)} rows")


    # Handle missing values
    numerical_cols = [
        'distance_f', 'purse_x', 'dollar_odds', 'weight', 'age',
        'recent_finish_pos_1', 'recent_lengths_back_finish_1', 'recent_purse_1',
        'recent_finish_pos_2', 'recent_lengths_back_finish_2', 'recent_purse_2',
        'recent_finish_pos_3', 'recent_lengths_back_finish_3', 'recent_purse_3',
        'recent_finish_pos_4', 'recent_lengths_back_finish_4', 'recent_purse_4',
        'recent_finish_pos_5', 'recent_lengths_back_finish_5', 'recent_purse_5'
    ]
    for col in features:
        if col in df.columns:
            # Use df[col].dtype instead of df[col].dtypes
            # Check if the column's dtype is a numerical type
            if pd.api.types.is_numeric_dtype(df[col]): # Corrected to check the column's dtype
                df[col] = df[col].fillna(-1)
            else:
                df[col] = df[col].fillna('unknown')
        else:
            df[col] = -1 if col in numerical_cols else 'unknown'




    # Normalize numerical features
    scaler = MinMaxScaler()
    for col in numerical_cols:
        if col not in df.columns:
            df[col] = -1
    df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

    # Create text sequences
    def create_text_sequence(row):
        text = (
            f"Horse: {row['horse']}, Jockey: {row['jockey']}, Trainer: {row['trainer']}, "
            f"Program: {row['program_number']}, Surface: {row['surface_x']}, "
            f"Distance: {row['distance_f']:.2f} furlongs, Purse: {row['purse_x']:.2f}, "
            f"Track: {row['track_code']}, Date: {row['race_date']}, Race: {row['race_number']}, "
            f"Type: {row['race_type']}, Weight: {row['weight']:.2f}, Age: {row['age']:.2f}, "
            f"Sex: {row['sex_code']}, Odds: {row['dollar_odds']:.2f}, "
            f"Recent Finish 1: {row['recent_finish_pos_1']:.2f}, Recent Lengths 1: {row['recent_lengths_back_finish_1']:.2f}, "
            f"Recent Purse 1: {row['recent_purse_1']:.2f}, "
            f"Recent Finish 2: {row['recent_finish_pos_2']:.2f}, Recent Lengths 2: {row['recent_lengths_back_finish_2']:.2f}, "
            f"Recent Purse 2: {row['recent_purse_2']:.2f}, "
            f"Recent Finish 3: {row['recent_finish_pos_3']:.2f}, Recent Lengths 3: {row['recent_lengths_back_finish_3']:.2f}, "
            f"Recent Purse 3: {row['recent_purse_3']:.2f}, "
            f"Recent Finish 4: {row['recent_finish_pos_4']:.2f}, Recent Lengths 4: {row['recent_lengths_back_finish_4']:.2f}, "
            f"Recent Purse 4: {row['recent_purse_4']:.2f}, "
            f"Recent Finish 5: {row['recent_finish_pos_5']:.2f}, Recent Lengths 5: {row['recent_lengths_back_finish_5']:.2f}, "
            f"Recent Purse 5: {row['recent_purse_5']:.2f}"
        )
        return text
    df['input_text'] = df.apply(create_text_sequence, axis=1)
    if not is_test and all(col in df.columns for col in targets):
        df['target_text'] = df.apply(
            lambda row: f"Finish: {row['official_finish']}, Speed: {row['speed_rating']}, Time: {row['win_time']}", axis=1
        )
    else:
        df['target_text'] = 'unknown'  # Placeholder for test data without targets

    # Preserve necessary columns
    columns_to_keep = ['input_text', 'target_text', 'horse', 'program_number', 'race_number'] + (targets if all(col in df.columns for col in targets) else [])
    return df[columns_to_keep], scaler # Modified to return necessary columns


# Step 2: Custom Dataset
class RaceDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.has_targets = all(col in df.columns for col in ['official_finish', 'speed_rating', 'win_time'])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        input_text = self.df.iloc[idx]['input_text']
        target_text = self.df.iloc[idx]['target_text']
        targets = self.df.iloc[idx][['official_finish', 'speed_rating', 'win_time']].values.astype(float) if self.has_targets else np.zeros(3)

        # Tokenize input
        input_encoding = self.tokenizer(
            input_text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        # Tokenize target (for decoder input)
        target_encoding = self.tokenizer(
            target_text,
            max_length=32,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encoding['input_ids'].squeeze(),
            'attention_mask': input_encoding['attention_mask'].squeeze(),
            'decoder_input_ids': target_encoding['input_ids'].squeeze(),
            'decoder_attention_mask': target_encoding['attention_mask'].squeeze(),
            'labels': torch.tensor(targets, dtype=torch.float) if self.has_targets else torch.zeros(3, dtype=torch.float)
        }

# Step 3: Custom BART Model for Regression
class BartForRegression(nn.Module):
    def __init__(self, bart_model):
        super(BartForRegression, self).__init__()
        self.bart = bart_model
        self.regression_head = nn.Linear(self.bart.config.d_model, 3)  # 3 outputs: official_finish, speed_rating, win_time

    def forward(self, input_ids, attention_mask, decoder_input_ids, decoder_attention_mask):
        outputs = self.bart(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask
        )
        last_hidden_state = outputs.last_hidden_state[:, 0, :]
        predictions = self.regression_head(last_hidden_state)
        return predictions

# Step 4: Training Loop
def train_model(model, train_loader, val_loader, device, epochs=5):
    optimizer = AdamW(model.parameters(), lr=5e-5)
    best_val_loss = float('inf')
    patience = 2
    patience_counter = 0

    for epoch in tqdm(range(epochs), desc = 'Epoch Training' ):
        model.train()
        train_loss = 0
        for k, batch in enumerate(train_loader):
            print('batch: ', k,'/',len(train_loader))
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            decoder_input_ids = batch['decoder_input_ids'].to(device)
            decoder_attention_mask = batch['decoder_attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            predictions = model(input_ids, attention_mask, decoder_input_ids, decoder_attention_mask)
            loss = nn.MSELoss()(predictions, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}")

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                decoder_input_ids = batch['decoder_input_ids'].to(device)
                decoder_attention_mask = batch['decoder_attention_mask'].to(device)
                labels = batch['labels'].to(device)

                predictions = model(input_ids, attention_mask, decoder_input_ids, decoder_attention_mask)
                loss = nn.MSELoss()(predictions, labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f"Epoch {epoch+1}, Val Loss: {val_loss:.4f}")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_bart_model.pt')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered")
                break

# Step 5: Evaluate on Test Set
def evaluate_model(model, test_file_or_df, tokenizer, scaler, device, is_external_test=False):
    # Load test data if a file path is provided
    if isinstance(test_file_or_df, str):
        test_df, _ = load_and_preprocess_data(test_file_or_df, is_test=True)
    else:
        test_df = test_file_or_df

    test_dataset = RaceDataset(test_df, tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=8)

    model.eval()
    predictions = []
    actuals = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            decoder_input_ids = batch['decoder_input_ids'].to(device)
            decoder_attention_mask = batch['decoder_attention_mask'].to(device)
            labels = batch['labels'].to(device)

            preds = model(input_ids, attention_mask, decoder_input_ids, decoder_attention_mask)
            predictions.append(preds.cpu().numpy())
            if test_dataset.has_targets:
                actuals.append(labels.cpu().numpy())

    predictions = np.vstack(predictions)
    output_file = 'external_test_predictions.csv' if is_external_test else 'internal_test_predictions.csv'

    if test_dataset.has_targets:
        actuals = np.vstack(actuals)
        mse = mean_squared_error(actuals, predictions, multioutput='raw_values')
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(actuals, predictions, multioutput='raw_values')

        print("\nTest Set Evaluation Metrics:")
        for i, metric in enumerate(['Official Finish', 'Speed Rating', 'Win Time']):
            print(f"{metric}:")
            print(f"  MSE: {mse[i]:.4f}")
            print(f"  RMSE: {rmse[i]:.4f}")
            print(f"  MAE: {mae[i]:.4f}")

        results = pd.DataFrame({
            'actual_finish': actuals[:, 0],
            'predicted_finish': predictions[:, 0],
            'actual_speed': actuals[:, 1],
            'predicted_speed': predictions[:, 1],
            'actual_time': actuals[:, 2],
            'predicted_time': predictions[:, 2],
            'horse': test_df['horse'],
            'program_number': test_df['program_number'],
            'race_number': test_df['race_number']
        })
    else:
        print("\nNo target columns in test data. Saving predictions only.")
        results = pd.DataFrame({
            'predicted_finish': predictions[:, 0],
            'predicted_speed': predictions[:, 1],
            'predicted_time': predictions[:, 2],
            'horse': test_df['horse'],
            'program_number': test_df['program_number'],
            'race_number': test_df['race_number']
        })

    results.to_csv(output_file, index=False)
    print(f"\nTest predictions saved to '{output_file}'")
    return predictions, actuals if test_dataset.has_targets else None

In [2]:
train_file = 'merged_race_data_20250509_modified.csv'
external_test_file = '/content/CDX0426_filtered.csv'

# Load and preprocess
df, scaler = load_and_preprocess_data(train_file)

Loaded data with 1757 rows and 122 columns
After dropping NaN targets: 1481 rows


In [3]:
train_val_df, test_df = train_test_split(df, test_size=0.15, random_state=77)
train_df, val_df = train_test_split(train_val_df, test_size=0.1765, random_state=77)  # 0.1765 ≈ 0.15/(1-0.15)
print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")


Train: 1035, Val: 223, Test: 223


In [4]:
  # Initialize tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
bart_model = BartModel.from_pretrained('facebook/bart-base')
model = BartForRegression(bart_model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


BartForRegression(
  (bart): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): La

In [5]:
train_dataset = RaceDataset(train_df, tokenizer)
val_dataset = RaceDataset(val_df, tokenizer)
test_dataset = RaceDataset(test_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [6]:
train_model(model, train_loader, val_loader, device, epochs=100)

Epoch Training:   0%|          | 0/100 [00:00<?, ?it/s]

batch:  0 / 17
batch:  1 / 17
batch:  2 / 17
batch:  3 / 17
batch:  4 / 17
batch:  5 / 17
batch:  6 / 17
batch:  7 / 17
batch:  8 / 17
batch:  9 / 17
batch:  10 / 17
batch:  11 / 17
batch:  12 / 17
batch:  13 / 17
batch:  14 / 17
batch:  15 / 17
batch:  16 / 17
Epoch 1, Train Loss: 3519.8296
Epoch 1, Val Loss: 2987.5911


Epoch Training:   1%|          | 1/100 [01:06<1:49:24, 66.30s/it]

batch:  0 / 17
batch:  1 / 17
batch:  2 / 17
batch:  3 / 17
batch:  4 / 17
batch:  5 / 17
batch:  6 / 17
batch:  7 / 17
batch:  8 / 17
batch:  9 / 17
batch:  10 / 17
batch:  11 / 17
batch:  12 / 17
batch:  13 / 17
batch:  14 / 17
batch:  15 / 17
batch:  16 / 17
Epoch 2, Train Loss: 2821.9818
Epoch 2, Val Loss: 2791.2872


Epoch Training:   2%|▏         | 2/100 [02:15<1:50:54, 67.90s/it]

...

batch:  0 / 17
batch:  1 / 17
batch:  2 / 17
batch:  3 / 17
batch:  4 / 17
batch:  5 / 17
batch:  6 / 17
batch:  7 / 17
batch:  8 / 17
batch:  9 / 17
batch:  10 / 17
batch:  11 / 17
batch:  12 / 17
batch:  13 / 17
batch:  14 / 17
batch:  15 / 17
batch:  16 / 17
Epoch 81, Train Loss: 91.8575


Epoch Training:  81%|████████  | 81/100 [1:34:58<20:58, 66.26s/it]

Epoch 81, Val Loss: 213.8422
batch:  0 / 17
batch:  1 / 17
batch:  2 / 17
batch:  3 / 17
batch:  4 / 17
batch:  5 / 17
batch:  6 / 17
batch:  7 / 17
batch:  8 / 17
batch:  9 / 17
batch:  10 / 17
batch:  11 / 17
batch:  12 / 17
batch:  13 / 17
batch:  14 / 17
batch:  15 / 17
batch:  16 / 17
Epoch 82, Train Loss: 93.0675


Epoch Training:  81%|████████  | 81/100 [1:36:00<22:31, 71.12s/it]

Epoch 82, Val Loss: 208.2762
Early stopping triggered


In [10]:
torch.save(model.state_dict(), 'best_bart_model_manually_epoch_81.pt') # When manually stop run this code

In [11]:
model.load_state_dict(torch.load('best_bart_model.pt'))
model.to(device)

BartForRegression(
  (bart): BartModel(
    (shared): BartScaledWordEmbedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): La

In [12]:
# Evaluate on internal test set
print("\nEvaluating on internal test set from training data...")
evaluate_model(model, test_df, tokenizer, scaler, device, is_external_test=False)



print("\nModel trained, evaluated, and predictions saved.")


Evaluating on internal test set from training data...

Test Set Evaluation Metrics:
Official Finish:
  MSE: 8.8168
  RMSE: 2.9693
  MAE: 2.4076
Speed Rating:
  MSE: 379.7609
  RMSE: 19.4875
  MAE: 15.0905
Win Time:
  MSE: 149.7183
  RMSE: 12.2359
  MAE: 6.6227

Test predictions saved to 'internal_test_predictions.csv'

Model trained, evaluated, and predictions saved.
